In [61]:
import polars as pl
from mimesis import Person, Address, Numeric
import xlsxwriter

In [20]:
peep = Person()
adrs = Address()
num = Numeric()

In [55]:
data = []
for _ in range(100_000):
    data.append({'name':peep.full_name(), 'address':adrs.address(), 'zip_cd':adrs.zip_code()
                 , 'state':adrs.state(), 'net_worth':num.integer_number(5000,50000)})

In [56]:
df = pl.DataFrame(data)

In [52]:
df.head(5)

name,address,zip_cd,state,net_worth
str,str,str,str,i64
"""Dominque Sloan…","""962 Auto Cross…","""87605""","""Pennsylvania""",46978
"""Glennie Ruiz""","""467 Minnesota …","""50005""","""Virginia""",24189
"""Theo Reese""","""896 Alhambra H…","""36359""","""Hawaii""",13333
"""Rogelio Conner…","""233 Rio Ranch""","""79718""","""New Hampshire""",49943
"""Kenia Mueller""","""779 Valdez Str…","""56450""","""Hawaii""",23592


In [57]:
res = df.group_by(pl.col('zip_cd')).agg(pl.col('name').n_unique().name.suffix("_cnt")
                                        ,pl.col('address').count().name.suffix("_cnt")
                                        ,pl.col('net_worth').sum().name.suffix("_tot")
                            )
print(res)

shape: (63_168, 4)
┌────────┬──────────┬─────────────┬───────────────┐
│ zip_cd ┆ name_cnt ┆ address_cnt ┆ net_worth_tot │
│ ---    ┆ ---      ┆ ---         ┆ ---           │
│ str    ┆ u32      ┆ u32         ┆ i64           │
╞════════╪══════════╪═════════════╪═══════════════╡
│ 95887  ┆ 4        ┆ 4           ┆ 79981         │
│ 78005  ┆ 1        ┆ 1           ┆ 13615         │
│ 88794  ┆ 1        ┆ 1           ┆ 49062         │
│ 40225  ┆ 1        ┆ 1           ┆ 31707         │
│ 06088  ┆ 1        ┆ 1           ┆ 5785          │
│ …      ┆ …        ┆ …           ┆ …             │
│ 90521  ┆ 2        ┆ 2           ┆ 74368         │
│ 46583  ┆ 1        ┆ 1           ┆ 47269         │
│ 19442  ┆ 1        ┆ 1           ┆ 48631         │
│ 32625  ┆ 2        ┆ 2           ┆ 72147         │
│ 71798  ┆ 2        ┆ 2           ┆ 51017         │
└────────┴──────────┴─────────────┴───────────────┘


In [62]:
res.write_parquet("./agg_pl.parquet")
res.write_excel("./test.xlsx")